# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow instructions and use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever,
to query against two pdf documents we will upload.

The architeture and data flow diagram below depicts the interaction among all components that comprise OpenAI Assistant APIs. Central to understand is the Threads and Runtime that executes anyschronously, adding and reading messages to the Threads.

For integrating the Assistants API:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools

Below we follow those steps to demonstrate how to integrate Assistants API, using function tool, to ask our Assistant to interact with an external web services, such
as Google Search, Weather Stacks, and OpenAI too.

This external service could be any external [API Webserivce](https://apilayer.com/)

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="images/assistant_ai_tools_code_interpreter.png">


## How to use Assistant API using Tools: Python code interpreter
In this example, we will use Python Code interpreter. That is,
our Assistant will be asked to analyse an uploaded file, generate an image,
and a python script. We can ask Assistant questions about the data file loaded, and it will generate a Python script to run it.

This is an example of how an Assistant can employ an external tool, such as a Python code interpreter.  Our query could be part of a larger application using LLM and Assitant to respond to user queries to generate Python scripts 
that can be used downstream.

Let's see how we can do it. The steps are not dissimilar to our
previous notebook. 

In [1]:
import warnings
import os
import json
import time

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
from typing import List, Dict, Any
from assistant_utils import print_thread_messages, upload_files, \
                            create_assistant_run 

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Assistant API calling for Anyscale Endpoints (which serves only OS models) is not yet aviable).

In [2]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
DOCS_TO_LOAD = ["docs/ray_meetups_data.csv"]

### Step 1: Create our knowledgebase
This entails uploading your files for the Assistant to use.

The Python interpreter will use these files to answer your user 
queries regarding the darta in the file

Upload the data file from your storage.

In [5]:
file_objects = upload_files(client, DOCS_TO_LOAD)
file_objects

[FileObject(id='file-NvpyZyVwrFT2IGKPUvoIKz0x', bytes=612, created_at=1703886366, filename='ray_meetups_data.csv', object='file', purpose='assistants', status='processed', status_details=None)]

In [6]:
# Extract file ids 
file_obj_ids = []
for f_obj in file_objects:
    file_obj_ids.append(file_objects[0].id)
file_obj_ids

['file-NvpyZyVwrFT2IGKPUvoIKz0x']

### Step 2: Create an Assistant 
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object. Supply the Assistant with a model to use, tools, i.e., Code Interpreter

In [7]:
assistant = client.beta.assistants.create(name="Data Analyst",
                                           instructions="""You are a knowledgeable chatbot trained to respond 
                                               inquires on documents CSV data files.
                                               Use a neutral, professional advisory tone, and only respond by consulting the 
                                               knowledge base or files you are granted access to. 
                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid
                                               I don't have access to that information.'""",
                                           model=MODEL,
                                           tools = [{"type": "code_interpreter"}],  # use the Code Interpreter tool
                                           file_ids=file_obj_ids # use these CSV files uploaded as part of your knowledge base
)                                        
assistant

Assistant(id='asst_jQDnJlKpUxc0NpJru3BnhCfP', created_at=1703886366, description=None, file_ids=['file-NvpyZyVwrFT2IGKPUvoIKz0x'], instructions="You are a knowledgeable chatbot trained to respond \n                                               inquires on documents CSV data files.\n                                               Use a neutral, professional advisory tone, and only respond by consulting the \n                                               knowledge base or files you are granted access to. \n                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid\n                                               I don't have access to that information.'", metadata={}, model='gpt-4-1106-preview', name='Data Analyst', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])

### Step 3: Create a thread 
As the diagram above shows, the Thread is the conversational object with which the Assistant runs will interact with, by fetching messages (queries) and putting messages (responses) to it. Think of a thread as a "conversation session" between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [8]:
thread = client.beta.threads.create()
thread

Thread(id='thread_sXIZbavYkmRqpHvTGwKfjpyk', created_at=1703886367, metadata={}, object='thread')

### Step 4: Add your message query to the thread for the Assistant

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""Show me the Ray meetup membership growth over the years as linear chart. Save 
    it as ray_growth_meeetup.png". Create two wide bar chartsfor the RSVPs and Attended respectively. 
    Use the x-axis as meetup dates and y-axis as meetup members. Plot bar charts in a stack manner into a single file. 
    Save it as rsvp_attended.png. Finally, generate the Python code to accomplish this task, and save as code_gen.py""",
)
message.model_dump()

{'id': 'msg_oCJxZ7kGjSCg3JoH3M53A5gl',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'Show me the Ray meetup membership growth over the years as linear chart. Save \n    it as ray_growth_meeetup.png". Create two wide bar charts for the RSVPs and Attended respectively. \n    Use the x-axis as meetup dates and y-axis as meetup members. Plot bar charts in a stack manner into a single file. \n    Save it as rsvp_attended.png. Finally, generate the Python code to accomplish this task, and save as code_gen.py'},
   'type': 'text'}],
 'created_at': 1703886367,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_sXIZbavYkmRqpHvTGwKfjpyk'}

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [10]:
instruction_msg = """Please address the user as Jules Dmatrix."""
run = create_assistant_run(client, assistant, thread, instruction_msg)
print(run.model_dump_json(indent=2))

{
  "id": "run_VySRlRoQPHgqvUkvvKzh29lo",
  "assistant_id": "asst_jQDnJlKpUxc0NpJru3BnhCfP",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": 1703886367,
  "expires_at": 1703886967,
  "failed_at": null,
  "file_ids": [
    "file-NvpyZyVwrFT2IGKPUvoIKz0x"
  ],
  "instructions": "Please address the user as Jules Dmatrix.",
  "last_error": null,
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "object": "thread.run",
  "required_action": null,
  "started_at": null,
  "status": "queued",
  "thread_id": "thread_sXIZbavYkmRqpHvTGwKfjpyk",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ]
}


### Step 6: Retrieve the status and loop until the Assistant run status is `completed.`

Loop until run status is **required_action**, which is a trigger notification to extract arguments generated by the LLM model and carry onto the next step: invoke the function with the generated arguments.

In [11]:
while True:
    time.sleep(5)
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.status)
    
    # If run is completed, get all the messages
    # on the thread, inserted by the Assistant's run
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id)

        # Loop through messages and print content based on role
        # and break out of the while loop
        print("\nFinal output from the Assistant run:")
        print_thread_messages(client, thread, content_value=False)        
        break
    else:
        print(f"Assistant run state: '{run_status.status}' ...")
        time.sleep(5)

in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
in_progress
Assistant run state: 'in_progress' ...
completed

Final output from the Assistant run:
ThreadMessage(id='msg_oE1a5ywQvASIMr5PyeZrrBwI', assistant_id='asst_jQDnJlKpUxc0NpJru3BnhCfP', content=[MessageContentText(text=Text(annotations=[TextAnnotationFilePath(end_index=198, fil

### Step 7: Extract generated files from the Code Interpreter

Partial code for iterating over messages and extracting files borrowed from [here](https://www.youtube.com/watch?v=vW4RSEB4Zzo&t=22s)

In [12]:
messages = client.beta.threads.messages.list(
            thread_id=thread.id)

for message in messages:
    print("-" * 50)
    # Print the role of the sender
    print(f"Role: {message.role}")

    # Process each content item in the message
    for content in message.content:
        # Check if the content is text
        if content.type == 'text':
            print(content.text.value)

            # Check and print details about annotations if they exist
            if content.text.annotations:
                for annotation in content.text.annotations:
                    print(f"Annotation Text: {annotation.text}")
                    print(f"File_Id: {annotation.file_path.file_id}")
                    annotation_data = client.files.content(annotation.file_path.file_id)
                    annotation_data_bytes = annotation_data.read()

                    # file_extension = annotation.text.split('.')[-1]
                    filename = annotation.text.split('/')[-1]

                    with open(f"{filename}", "wb") as file:
                        file.write(annotation_data_bytes)
            else:
                print("No annotations found")

        # Check if the content is an image file and print its file ID and name
        elif content.type == 'image_file':
            print(f"Image File ID: {content.image_file.file_id}")
            image_data = client.files.content(content.image_file.file_id)
            image_data_bytes = image_data.read()

            with open(f"{content.image_file.file_id}.png", "wb") as file:
                file.write(image_data_bytes)

    # Print a horizontal line for separation between messages
    print("-" * 50)
    print('\n')

--------------------------------------------------
Role: assistant
The Python code has been generated to accomplish the visualization task and saved as "code_gen.py". You can download the code using the following link:

- [code_gen.py](sandbox:/mnt/data/code_gen.py)

In this code, replace `'path_to_your_file.csv'` with the actual path to your data file when running it in your environment. If you have any further questions or need assistance with anything else, let me know!
Annotation Text: sandbox:/mnt/data/code_gen.py
File_Id: file-VuzJreEVfhLC8BxWSgJI0mpg
--------------------------------------------------


--------------------------------------------------
Role: assistant
The visualizations have been created and saved as requested. Here are the links to download the images:

- **Ray Meetup Membership Growth Chart**: [ray_growth_meetup.png](sandbox:/mnt/data/ray_growth_meetup.png)
- **RSVPs and Attended Stacked Bar Charts**: [rsvp_attended.png](sandbox:/mnt/data/rsvp_attended.png)

N

In [13]:
# Delete the assistant. Optionally, you can delete any files
# associated with it that you have uploaded onto the OpenAI platform

response = client.beta.assistants.delete(assistant.id)
print(response)

for file_id in file_obj_ids:
    print(f"deleting file id: {file_id}...")
    response = client.files.delete(file_id)
    print(response)

AssistantDeleted(id='asst_jQDnJlKpUxc0NpJru3BnhCfP', deleted=True, object='assistant.deleted')
deleting file id: file-NvpyZyVwrFT2IGKPUvoIKz0x...
FileDeleted(id='file-NvpyZyVwrFT2IGKPUvoIKz0x', deleted=True, object='file')
